In [8]:
import cv2
import numpy as np
import os
from imutils import paths
import time

In [4]:
class cropped_image():
    
    def __init__(self, imagepath):
        self.cropping = False
        self.path = imagepath
        self.roi = []
 
        self.x_start, self.y_start, self.x_end, self.y_end = 0, 0, 0, 0
 
        self.image = cv2.imread(imagepath)
        self.oriImage = self.image.copy()
 
    def mouse_crop(self, event, x, y, flags, param):

        # if the left mouse button was DOWN, start RECORDING
        # (x, y) coordinates and indicate that cropping is being
        if event == cv2.EVENT_LBUTTONDOWN:
            self.x_start, self.y_start, self.x_end, self.y_end = x, y, x, y
            self.cropping = True

        # Mouse is Moving
        elif event == cv2.EVENT_MOUSEMOVE:
            if self.cropping == True:
                self.x_end, self.y_end = x, y

        # if the left mouse button was released
        elif event == cv2.EVENT_LBUTTONUP:
            # record the ending (x, y) coordinates
            self.x_end, self.y_end = x, y
            self.cropping = False # cropping is finished

            refPoint = [(self.x_start, self.y_start), (self.x_end, self.y_end)]

            if len(refPoint) == 2: #when two points were found
                self.roi = self.oriImage[refPoint[0][1]:refPoint[1][1], refPoint[0][0]:refPoint[1][0]]
                try:
                    cv2.imshow("Cropped", self.roi)
                except:
                    pass
                    
    def capture(self):
        cv2.namedWindow(self.path)
        cv2.setMouseCallback(self.path, self.mouse_crop)

        while True:

            i = self.image.copy()

            if not self.cropping:
                cv2.imshow(self.path, self.image)

            elif self.cropping:
                cv2.rectangle(i, (self.x_start, self.y_start), (self.x_end, self.y_end), (255, 0, 0), 2)
                cv2.imshow(self.path, i)



            k = cv2.waitKey(1) & 0xFF
            
            if k == ord('s'):
                try:
                    cv2.imwrite(self.path, self.roi)
                except:
                    pass
            if k == ord('q'):
                break

        # close all open windows
        cv2.destroyAllWindows()
        cv2.waitKey(1)

In [5]:
def cv_disp_img(img, title="image"):
    while True:
        cv2.imshow(title, img)
        k = cv2.waitKey(1)

        if k & 0xFF == ord('q'):
            # q key pressed so quit
            print("Quitting...")
            break

    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [9]:
def capture_faces(name):
    epoch = time.time()
    directory = os.path.join(os.path.abspath("./Dataset/Train"))
    
    cap_cnt = -1
    for file in os.listdir(directory):
        if name in file:
            temp = int(file.split(".")[1])
            if temp > cap_cnt:
                cap_cnt = temp
            
    cap_cnt += 1
    print("previous pictures: ", cap_cnt-1)
    
    cv2.namedWindow(name)
    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
#     eyes_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
    
    
    
    webcam = cv2.VideoCapture(0)
    
    success = webcam.isOpened()
    if success == False:
        print('Error: Camera could not be opened')
    else:
        print('Success: Grabbed the camera')
    while True:
        ret, frame = webcam.read()
        save_frame = frame.copy()
        
        faces = face_cascade.detectMultiScale(frame, scaleFactor=1.4, minNeighbors=6)
        for (x,y,w,h) in faces:
#             eyes = eyes_cascade.detectMultiScale(frame[y:y+h, x:x+w])
            if (time.time() - epoch) >= 3:
                cap_name = f"{name}.{cap_cnt}.png"
                p = 0
                cv2.imwrite(os.path.join(directory,cap_name), save_frame[y-p+1:y+h+p, x-p+1:x+w+p])            
                print(f"Saving {cap_name}!")
                cap_cnt += 1
                epoch = time.time()
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 1);
#             for (ex, ey, ew, eh) in eyes:
#                 cv2.rectangle(frame, (x+ex, y+ey), (x+ex+ew, y+ey+eh), (255, 255, 255), 1)
        cv2.imshow(name, frame)
        if not ret:
            break
        # Monitor keystrokes
        k = cv2.waitKey(1)

        if k & 0xFF == ord('q'):
            print("Quitting...")
            break
#         elif k & 0xFF == ord('c'):
#             cap_name = f"{name}.{cap_cnt}.png"
#             p = 0
#             cv2.imwrite(os.path.join(directory,cap_name), save_frame[y-p+1:y+h+p, x-p+1:x+w+p])            
#             print(f"Saving {cap_name}!")
#             cap_cnt += 1
            
    webcam.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [10]:
capture_faces("Mom")

previous pictures:  23
Success: Grabbed the camera
Saving Mom.24.png!
Quitting...


In [11]:
imagePaths = np.sort(list(paths.list_images("./Dataset/Train/")))
for imagePath in imagePaths:
    if "Mom" in imagePath:
        cropped_image(imagePath).capture()

KeyboardInterrupt: 